In [3]:
import pandas as pd
from catboost import CatBoostClassifier, Pool

In [5]:
df = pd.read_pickle("../sybil_features_all_addresses.pickle")
df

,tx_count,active_days,lifetime_days,mean_tx_interval,median_tx_interval,std_tx_interval,max_tx_interval,fast_tx_ratio,short_interval_ratio,tx_per_day,...,peak_hour,peak_hour_ratio,hour_entropy,weekend_tx_ratio,unique_tx_types,tx_type_entropy,confirmations_mean,confirmations_std,low_conf_ratio,address
0,20,7,77.844190,3.539862e+05,2924.0,7.086010e+05,2544743.0,0.200000,0.250000,2.857143,...,4,0.350000,1.915765,0.050000,3,0.730588,1.501222e+08,6.539440e+07,0.250000,0xd6614286c4fdbdd4d39f153fc13b53368b3bd0b8
1,24,6,272.470127,1.023540e+06,170.0,4.696790e+06,22562155.0,0.291667,0.416667,4.000000,...,10,0.416667,1.517852,0.416667,3,1.063521,1.445570e+08,9.189451e+07,0.250000,0x01e1b85434391d6965de8b35acbb8884059afae7
2,145,44,478.125579,2.868753e+05,268.5,1.085975e+06,10794773.0,0.082759,0.172414,3.295455,...,18,0.186207,2.632444,0.558621,3,0.649059,1.738306e+08,7.258849e+07,0.248276,0xf0c58b786a35c767682c907a82ba5455490b3591
3,54,17,173.653148,2.830874e+05,28.0,1.052020e+06,6980659.0,0.370370,0.518519,3.176471,...,8,0.240741,2.271567,0.351852,3,0.979288,2.105488e+08,8.317516e+07,0.240741,0xed313446dbfbec87900fe0a7dfde688f744a3819
4,308,42,533.474502,1.501374e+05,47.0,6.986825e+05,7339859.0,0.275974,0.545455,7.333333,...,5,0.250000,2.317984,0.577922,3,0.909097,1.859688e+08,8.184623e+07,0.246753,0xdb686a9b77bc516a5fce6307a6ca58f6cbde1e3a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56891,596,73,267.778924,3.888420e+04,91.0,2.005161e+05,2985210.0,0.208054,0.439597,8.164384,...,19,0.184564,2.395846,0.337248,3,0.961837,1.434240e+08,6.525561e+07,0.250000,0xe0768e83840e09dc85c53833a9d3b4d16185fcfb
56892,108,26,377.828843,3.050880e+05,191.0,9.954825e+05,7303444.0,0.240741,0.425926,4.153846,...,12,0.351852,2.156182,0.157407,3,0.876516,1.849920e+08,6.498987e+07,0.250000,0xdb433bc5d23f2cdbdfcd96a0dcb73891b5f122d6
56893,240,56,810.174595,2.928832e+05,39.0,1.524632e+06,21399012.0,0.383333,0.533333,4.285714,...,7,0.150000,2.651510,0.345833,3,0.929510,2.031344e+08,8.090620e+07,0.250000,0x1d5e033ae2aa830bf7621c6349e0a00e3e071402
56894,434,67,148.937326,2.971867e+04,45.0,9.770210e+04,1107368.0,0.301843,0.534562,6.477612,...,8,0.135945,2.675979,0.278802,3,0.872723,1.224184e+08,3.990044e+07,0.251152,0xe98f0276a5f59c2b54886a17b0a2fac717915510


In [8]:
X = df.drop(columns=["address"])

In [10]:
model = CatBoostClassifier()
model.load_model("catboost_info/fold-0/model")

preds = model.predict(Pool(data=X), prediction_type="Probability")[:, 1]

for i in range(1, 5):
    model = CatBoostClassifier()
    model.load_model(f"catboost_info/fold-{i}/model")

    preds += model.predict(Pool(data=X), prediction_type="Probability")[:, 1]
    preds

preds = preds / 5

mapper = {
    0: "A",  # <25 %
    1: "B",  # [25%;50%)
    2: "C",  # [50%; 75%)
    3: "D",
}  # >=75%

df["our_rating"] = [
    mapper[int(x > 0.024642) + int(x > 0.097071) + int(x > 0.176236)] for x in preds
]
df

,tx_count,active_days,lifetime_days,mean_tx_interval,median_tx_interval,std_tx_interval,max_tx_interval,fast_tx_ratio,short_interval_ratio,tx_per_day,...,peak_hour_ratio,hour_entropy,weekend_tx_ratio,unique_tx_types,tx_type_entropy,confirmations_mean,confirmations_std,low_conf_ratio,address,our_rating
0,20,7,77.844190,3.539862e+05,2924.0,7.086010e+05,2544743.0,0.200000,0.250000,2.857143,...,0.350000,1.915765,0.050000,3,0.730588,1.501222e+08,6.539440e+07,0.250000,0xd6614286c4fdbdd4d39f153fc13b53368b3bd0b8,A
1,24,6,272.470127,1.023540e+06,170.0,4.696790e+06,22562155.0,0.291667,0.416667,4.000000,...,0.416667,1.517852,0.416667,3,1.063521,1.445570e+08,9.189451e+07,0.250000,0x01e1b85434391d6965de8b35acbb8884059afae7,C
2,145,44,478.125579,2.868753e+05,268.5,1.085975e+06,10794773.0,0.082759,0.172414,3.295455,...,0.186207,2.632444,0.558621,3,0.649059,1.738306e+08,7.258849e+07,0.248276,0xf0c58b786a35c767682c907a82ba5455490b3591,B
3,54,17,173.653148,2.830874e+05,28.0,1.052020e+06,6980659.0,0.370370,0.518519,3.176471,...,0.240741,2.271567,0.351852,3,0.979288,2.105488e+08,8.317516e+07,0.240741,0xed313446dbfbec87900fe0a7dfde688f744a3819,D
4,308,42,533.474502,1.501374e+05,47.0,6.986825e+05,7339859.0,0.275974,0.545455,7.333333,...,0.250000,2.317984,0.577922,3,0.909097,1.859688e+08,8.184623e+07,0.246753,0xdb686a9b77bc516a5fce6307a6ca58f6cbde1e3a,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56891,596,73,267.778924,3.888420e+04,91.0,2.005161e+05,2985210.0,0.208054,0.439597,8.164384,...,0.184564,2.395846,0.337248,3,0.961837,1.434240e+08,6.525561e+07,0.250000,0xe0768e83840e09dc85c53833a9d3b4d16185fcfb,A
56892,108,26,377.828843,3.050880e+05,191.0,9.954825e+05,7303444.0,0.240741,0.425926,4.153846,...,0.351852,2.156182,0.157407,3,0.876516,1.849920e+08,6.498987e+07,0.250000,0xdb433bc5d23f2cdbdfcd96a0dcb73891b5f122d6,B
56893,240,56,810.174595,2.928832e+05,39.0,1.524632e+06,21399012.0,0.383333,0.533333,4.285714,...,0.150000,2.651510,0.345833,3,0.929510,2.031344e+08,8.090620e+07,0.250000,0x1d5e033ae2aa830bf7621c6349e0a00e3e071402,B
56894,434,67,148.937326,2.971867e+04,45.0,9.770210e+04,1107368.0,0.301843,0.534562,6.477612,...,0.135945,2.675979,0.278802,3,0.872723,1.224184e+08,3.990044e+07,0.251152,0xe98f0276a5f59c2b54886a17b0a2fac717915510,A
